In [2]:
import numpy as np
import scipy
import pandas as pd
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL

In [3]:
%matplotlib widget
plt.rcParams["figure.autolayout"] = True
plt.rcParams["figure.figsize"] = [7,5]

# Data Cleaning

In [4]:
os.getcwd()

'/Users/aprilnellis/Desktop/recession-regression/Data'

In [ ]:
original = pd.read_csv('BLS_Employment.csv')
original

In [ ]:
original['Month'] = pd.to_datetime(original['Month'])
original

In [ ]:
#employ_diff = original.diff(axis = 0)
employ_diff = pd.read_csv('BLS_EmployDelta.csv')
employ_diff

In [ ]:
employ_diff.drop(0, axis=0, inplace = True)

In [ ]:
#employ_diff['Month'] = original['Month']
#employ_diff

In [ ]:
covid_lockdown = original[197:203]
covid_lockdown

In [15]:
original.drop(np.arange(198, 202), axis=0, inplace = True)

In [16]:
original.reset_index(drop = True, inplace = True)

In [ ]:
employ_diff.iloc[:,2:] = employ_diff.iloc[:, 2:]/original.iloc[:-1, 1:]

In [22]:
employ_diff.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [24]:
diff

,Month,Total nonfarm,Total private,Mining and logging,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Utilities,Information,Financial activities,Professional and business services,Private education and health services,Leisure and hospitality,Other services,Government,Federal government,State government,Local government
0,2003-10-01,0.000000,0.001223,0.005263,0.000147,-0.000906,-0.000054,0.001254,0.002210,-0.000174,-0.002214,-0.001235,0.001366,0.003495,0.004178,-0.000185,0.002511,-0.002546,0.001406,0.003920
1,2003-11-01,0.000000,0.000606,0.005236,0.001769,-0.001256,0.000379,-0.001607,0.000839,-0.002618,-0.001585,-0.000865,0.002356,0.002125,0.002039,0.000370,-0.001067,-0.005106,0.000201,-0.000723
2,2003-12-01,0.000000,0.000936,-0.001736,0.004562,-0.001118,0.002202,-0.000168,-0.003162,-0.001225,0.000000,-0.000866,0.001670,0.002238,0.002931,-0.000185,0.000511,0.004399,-0.001403,0.000434
3,2004-01-01,0.000000,0.001632,0.001739,0.003076,-0.000699,0.002054,0.002892,0.007233,-0.001051,-0.002857,0.000248,0.001791,0.001881,0.002110,0.000370,-0.000371,-0.002920,-0.003212,0.001157
4,2004-02-01,0.000000,0.000412,0.001736,-0.001460,-0.000770,0.001025,-0.000060,0.001097,-0.002631,0.000000,0.000000,0.000801,0.001056,0.002106,0.000000,0.000557,-0.000366,0.000403,0.000795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,2023-05-01,0.001805,0.001916,0.004687,0.003164,-0.000308,0.001273,0.001346,0.002768,0.001259,-0.001292,0.001425,0.001959,0.003499,0.001694,0.002225,0.001150,0.003093,0.002691,0.000207
232,2023-06-01,0.000673,0.000645,-0.001555,0.003659,0.000308,-0.001107,-0.001460,-0.002834,0.001437,-0.003235,0.000219,-0.000043,0.003130,0.001570,0.001025,0.000840,0.002055,0.002876,-0.000138
233,2023-07-01,0.001512,0.001087,0.003115,0.001508,-0.000154,0.002547,0.000812,-0.001012,-0.002691,-0.006167,0.001642,-0.001260,0.004108,0.002292,0.000682,0.004018,0.003419,0.002294,0.004761
234,2023-08-01,0.001452,0.001325,0.000000,0.004519,0.000847,0.000000,0.000026,-0.002816,0.004317,-0.006858,0.000219,0.000479,0.003816,0.002647,0.002387,0.002199,0.003748,0.004578,0.001030


In [25]:
diff.to_csv('BLS_EmployDelta.csv')

# Simple Predictions

In [ ]:
employ_diff = pd.read_csv('BLS_EmployDelta.csv')

In [ ]:
employ_diff.info()

In [ ]:
employ_diff

In [ ]:
plt.close()
plt.plot(employ_diff['Month'], employ_diff['Construction'])
plt.show()

## Check for seasonality trend:

In [ ]:
seasonal = STL(employ_diff['Construction'], period=12)
res = seasonal.fit()
fig = res.plot()

I don't observe any particularly notable yearly trends!

## Try simple regression fit